In [238]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib as plt

In [239]:
df=pd.read_excel('Fashion - Assignment Data.xlsx')

In [334]:
df=df[df['Product_ID']!='undefined']
df['Invoice'] = df.index    #Adding a unique number as the invoice number to access each row uniquely
df.head()

,timestamp,user ID,ip_address,Product_Name,Product_ID,Is_First_Order,User_Gender,Payment_Type,Number_of_Products,Order_Coupon_Code,City,Country_Province,User_Birthday,Country,Revenue,Invoice
0,2017-10-21 20:52:31,0,49.227.243.31,Kendrick,SE043SH31VBS,0,female,cc@braintree,1,YAY15,Kaitaia,NaN,1993-10-05 00:00:00,NZ,212.5,0
1,2017-10-21 20:55:12,0,111.220.172.119,Smartwatch Bradshaw Gold,MI329AC90OUB,1,male,pbi@afterpay,1,NaN,Shepparton,VIC,NaN,AU,526.36,1
2,2017-10-21 20:56:41,0,210.84.59.179,Classic Slides,SA849SH69SAK,1,female,cc@braintree,1,NaN,Essendon,VIC,NaN,AU,77.23,2
3,2017-10-21 20:59:27,0,59.167.79.119,Bonaire Flared Sleeve Tunic,SH045AA22AGR,0,female,pbi@afterpay,1,NaN,Karabar,NSW,NaN,AU,190.91,3
4,2017-10-21 21:00:56,0,1.129.107.188,Tall Tales Man Style Pants,MA146AA45RAK,0,female,cc@braintree,1,NaN,Mosman,NSW,NaN,AU,204.09,4


In [335]:
df_affinity=df[['Invoice','Product_Name']]
print(len(df_affinity))
# Explode/Split column into multiple rows
df_affinity1 = \
(df_affinity.set_index(df_affinity.columns.drop('Product_Name',1).tolist())
   .Product_Name.str.split(',', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Product_Name'})
   .loc[:, df_affinity.columns]
)
df_affinity=df[['Invoice','Number_of_Products']]


49984


In [337]:
# df_affinity1['Quantity']=0
# df_affinity.head(50)
my_string=df_affinity.at[6,'Number_of_Products'].split(",")
# my_list = my_string.split(",")
my_list = [ int(x) for x in my_string ]
print(my_list)

[1, 1]


In [339]:
ind=0
for i in df_affinity.index:
    if type(df_affinity.at[i,'Number_of_Products']) is int:
        df_affinity1.at[ind,'Quantity']=df_affinity.at[i,'Number_of_Products']
        ind+=1
    elif type(df_affinity.at[i,'Number_of_Products']) is str:
        quan_list=df_affinity.at[i,'Number_of_Products'].split(",")
        quan_list = [ int(x) for x in quan_list ]
        for each in quan_list:
            df_affinity1.at[ind,'Quantity']=each
            ind+=1     
        
        

In [346]:
df_affinity1['Product_Name'] = df_affinity1['Product_Name'].str.strip()
df_affinity1.dropna(axis=0, subset=['Invoice'], inplace=True)
df_affinity1['Invoice'] = df_affinity1['Invoice'].astype('str')

In [347]:
df_affinity1.shape

(97903, 3)

In [348]:
df_affinity2 = (df_affinity1
                  .groupby(['Invoice', 'Product_Name'])['Quantity']
                  .sum().unstack().reset_index().fillna(0)
                  .set_index('Invoice'))

In [351]:
df_affinity2.head(3)

Product_Name,,"""Danny"" ChattyFeet Novelty Socks","""I"" Low Rise Logo Elastic WB Ankle Leggings","""Petite"" Toppy Mirror Block Heel Sandals","""Petite"" Tatum Diamante Strappy Heel Sandals","""Petite"" Totem Metallic Block Heel Sandals","""The New Guard""","""The New Guard"" Muscle Tank",'47 MVP,0-Size Jacket,...,adizero Backpack,pureBOOST Xpose - Women's,squid,x Teasy - New Low,Éclair Blouse,Éclair Dress,Édouard Tee,Émile Hooded Pullover Sweater,​Pro Indy Cooling Light Support Sports B,​Pro Indy Cooling Light Support Sports Bra
Invoice,,,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [354]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

aff_sets = df_affinity2.applymap(encode_units)

In [369]:
frequent_itemsets = apriori(aff_sets, min_support=0.0009, use_colnames=True)


In [370]:
print(frequent_itemsets)

      support                           itemsets
0    0.002822                (2750 Cotu Classic)
1    0.000981                        (2750 Lamé)
2    0.001101              (3-Pack Simple Socks)
3    0.001721  (3S Perf Crew Half Cushioned 3PP)
4    0.000981               (7" Distance Shorts)
..        ...                                ...
205  0.004123           (Women's Tubular Shadow)
206  0.001101                        (World Tee)
207  0.001201                   (X_Plr - Unisex)
208  0.002081            (Xtreme Elongated Tank)
209  0.001201                     (Zaina Wedges)

[210 rows x 2 columns]


In [378]:
rules = association_rules(frequent_itemsets, min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
